A notebook to handle figures for the (revisions of) the yoder et al. 2016 Global/Nepal ETAS paper.

In [ ]:
#
%load_ext autoreload
%autoreload 2
%matplotlib inline

import datetime as dtm
import matplotlib.dates as mpd
import pytz
tzutc = pytz.timezone('UTC')

#import operator
import math
import random
import numpy
import scipy
import scipy.optimize as spo
import itertools
import sys
#import scipy.optimize as spo
import os
import operator
#from PIL import Image as ipp
import multiprocessing as mpp
#
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
#import functools
#
#import shapely.geometry as sgp
#
from mpl_toolkits.basemap import Basemap as Basemap
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from geographiclib.geodesic import Geodesic as ggp
#
#import ANSStools as atp
from yodiipy import ANSStools as atp
#
import contours2kml
import globalETAS as gep
from eq_params import *
#
import random
import geopy
#
#colors_ =  mpl.rcParams['axes.color_cycle']
colors_ = ['b', 'g', 'r', 'c', 'm', 'y', 'k']		# make sure these are correct...
#
emc = {'lat':32.13, 'lon':-115.30, 'event_date':dtm.datetime(2010,4,4,22,40,41, tzinfo=pytz.timezone('UTC'))}
nepal_epi_lon = 84.698
nepal_epi_lat = 28.175

#
# set up dict objects for different earthquakes. maybe from eq_params.py ?
param_keys = ['lat_center', 'lon_center', 'to_dt', 'mc', 'mc_auto', 'cat_len_plus']
nepal_params = {key:val for key,val in zip(param_keys, [nepal_epi_lat, nepal_epi_lon, 
                                                        dtm.datetime(2015,4,30, tzinfo=pytz.timezone('UTC')),
                                                        4.0, 4.5, 220])}
emc_params =   {key:val for key,val in zip(param_keys, [emc['lat'], emc['lon'],
                                                      emc['event_date'] + dtm.timedelta(days=5), 2.5, 5.5, 220])}

### some random geopy bits:
  - easy syntax for great-circle or vincenty (more precise, iterative model) geodetic distances
  - forward and reverse geocoding
  

In [ ]:
from geopy.distance import vincenty
from geopy.distance import great_circle
newport_ri = (41.49008, -71.312796)
cleveland_oh = (41.499498, -81.695391)
print(vincenty(newport_ri, cleveland_oh).miles)
print(great_circle(newport_ri, cleveland_oh).miles)

from geopy.geocoders import Nominatim
geolocator = Nominatim()
#location = geolocator.geocode("175 5th Avenue NYC")
location = geolocator.geocode("509 S Waiola Ave, La Grange IL")
print(location.address)
print(location.point)

### ETAS and ROC figures
  - ETAS map
    - include recent (shortly before forecast date) earthquakes
  - ROC analysis
  
  Also include:
    - ETAS forecast catalog $t<t_0$
    - ETAS test catalog $t>t_0$
    - ETAS data.xyz file
    - Image(s)
    - KML
    - ROC table(s)
  
  For now, copy the make_Nepal_figs from nepal_figs.py. we'll move this to the original, or a new, module when we work out the bugs.

In [ ]:
from nepal_figs import *

class nepal__ROC_script(object):
    def __init__(self, fignum=0, mcs = [4., 5., 6., 7.], n_cpu=None):
        # this needs to be rewritten a bit to:
        # 1) use the same color for each magnitude
        # 2) should probably use the roc_generic class; see _rocs3()
        #
        # full, one stop shopping script for nepal ROC analysis.
        #
        # first, get nepal ETAS objects:
        etas_fc, etas_test = etas_analyzer.nepal_etas_roc()
        test_catalog = etas_test.catalog
        #
        x0 = nepal_epi_lon
        y0 = nepal_epi_lat
        mag=7.8
        L_r = .5*10**(.5*mag - 1.76)
        xyz = etas_fc.ETAS_array
        #
        # now, replace all of this "get x,y and ROC" stuff with the optimizers.roc_tools equivalents.
        roc_xyz_handler = ROC_xyz_handler(xyz, list(zip(test_catalog['lon'], test_catalog['lat'], test_catalog['mag'])))
        # ----- 27 july stop.
        #
        X_set = sorted(list(set(xyz['x'])))
        Y_set = sorted(list(set(xyz['y'])))
        nx = len(X_set)
        ny = len(Y_set)
        get_site = lambda x,y: int(round((x-etas_fc.lons[0]+.5*etas_fc.d_lon)/etas_fc.d_lon)) + int(round((y-etas_fc.lats[0]+.5*etas_fc.d_lat)/etas_fc.d_lat))*nx
        #
        xyz_r = xyz.copy()
        for j,(x,y,z) in enumerate(xyz_r):
            xyz_r['z'][j]=1./(dist_to(x,y,x0,y0) + .5*L_r)
        #
        Zs = sorted(list(xyz['z'].copy()))
        Zs_r = sorted(list(xyz_r['z'].copy()))
        #
        eq_site_zs = [[xyz['z'][get_site(eq['lon'], eq['lat'])], eq['mag']] for eq in test_catalog]
        #
        plt.figure(fignum)
        plt.clf()
        plt.plot(range(2), range(2), ls='--', color='m', lw=3., alpha=.75, zorder=2)
        FHs = {}        # we'll use mc as a key, FH as a val: {mc:[FH]...}
        for j,mc in enumerate(mcs):
            clr = colors_[j%len(colors_)]
            roc = roc_generic.ROC_mpp(n_procs=n_cpu, Z_events=[z for z,m in eq_site_zs if m>=mc], Z_fc=Zs, h_denom=None, f_denom=None, f_start=0, f_stop=None)
            a=roc.calc_roc()        # no parameters, and in fact no return value, but it's never a bad idea to leave a place-holder for one.
            #
            plt.plot(roc.F, roc.H, ls='-', color=clr, marker='', lw=2.5, label='$m_c=%.2f$' % mc)
            #FHs[mc]=[[f,h] for f,h in zip(roc.F, roc.H)]
            #
            roc = roc_generic.ROC_mpp(n_procs=n_cpu, Z_events=[z for z,m in eq_site_zs if m>=mc], Z_fc=Zs_r, h_denom=None, f_denom=None, f_start=0, f_stop=None)
            roc.calc_roc()
            plt.plot(roc.F, roc.H, ls='--', color=clr, marker='', lw=2.5, label='$m_c=%.2f$' % mc)
            #
            plt.show()    # just in case...
    
    
        #ROC_n = roc_normalses(etas_fc, test_catalog=None, to_dt=None, cat_len=120., mc_rocs=[4.5, 5.0, 6.0, 7.0], fignum=fignum, do_clf=True)
        #
        # now, make a toy catalog:
        #etas_toy = Toy_etas_invr(etas_in=etas_fc, mainshock={'mag':7.3, 'lon':84.698, 'lat':28.175})
        #
        #ROC_t = roc_normalses(etas_toy, test_catalog=None, to_dt=None, cat_len=120., mc_rocs=[4.5, 5.0, 6.0, 7.0], fignum=fignum, do_clf=False, roc_ls='--')
        #
        # now, some random catalogs:
        for j in range(25):
            this_etas = etas_analyzer.Toy_etas_random(etas_in=etas_fc)
            FH = etas_analyzer.roc_normal(this_etas, fignum=None)
            plt.plot(*zip(*FH), marker='.', ls='', alpha=.6)
        #
        self.__dict__.update(locals())
        #


In [ ]:
import global_etas_auto
#Location
#    18.515°N 145.529°E

aa=global_etas_auto.auto_etas(lon_center=145.529, lat_center=18.515, d_lat_0=1., d_lon_0=1., dt_0=30, Lr_factor=5.)


In [ ]:
aa.make_etas_contour_map()

